In [1]:
import requests
import time

API_BASE_URL = 'https://rddc.tsinghua-gd.org'
# API endpoints
API_SUBMIT = '/api/bio/aav'
API_TASK = '/api/bio/aav/tasks'

In [2]:
# example
data = {
    'sereotype': 'aav2',
    'sequences': [
        {'name': 'sequence 1', 'sequence': 'TTTGATTTGGCGCTTACTAATACG'},
        {'name': 'sequence 2', 'sequence': 'CCTGGGACTTCTTTTCTGGTG'},
        {'name': 'sequence 3', 'sequence': 'GGTTCGACTAGGTGTCCGGATTCG'},
        {'name': 'sequence 4', 'sequence': 'CCTATTCGTGATGGTGGTATGATGATG'},
        {'name': 'sequence 5', 'sequence': 'CCTCCTACGTTGGAGCAGCGGCCTTTTGTT'}
    ]
}

In [10]:
# Submit the data to the server
response = requests.post(f'{API_BASE_URL}{API_SUBMIT}', json=data)

# Check if submission was successful
response.raise_for_status()

# Retrieve the task ID from the response
task_id = response.text.strip()
print(f"✅ Task submitted successfully. Task ID: {task_id}")

# Poll the server every second for up to 100 seconds
for attempt in range(100):
    time.sleep(1)  # Wait before polling
    task_response = requests.post(f'{API_BASE_URL}{API_TASK}', json={"id": task_id})
    task_response.raise_for_status()
    
    result_json = task_response.json()
    status = result_json.get("status")

    if status == "SUCCESS":
        print("🎉 Task completed successfully.\n")
        result = result_json["data"]
        
        print(f"{'ID':<20}{'Sequence':<35}{'Score':>10}")
        print("-" * 70)
        for item in result:
            print(f"{item['id']:<20}{item['sequence']:<35}{float(item['score']):>10.3f}")
        break

    elif status == "FAILE":
        raise Exception("❌ Task failed on the server side.")

    else:
        print(f"⏳ Attempt {attempt + 1}/100: Task is still {status}...")

✅ Task submitted successfully. Task ID: a3374ff6c01a4896b1b41190950553fc
⏳ Attempt 1/100: Task is still RUNNING...
⏳ Attempt 2/100: Task is still RUNNING...
🎉 Task completed successfully.

ID                  Sequence                                Score
----------------------------------------------------------------------
sequence 1          TTTGATTTGGCGCTTACTAATACG                1.884
sequence 2          CCTGGGACTTCTTTTCTGGTG                  -1.539
sequence 3          GGTTCGACTAGGTGTCCGGATTCG                0.026
sequence 4          CCTATTCGTGATGGTGGTATGATGATG             1.832
sequence 5          CCTCCTACGTTGGAGCAGCGGCCTTTTGTT          0.205
